In [1]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

warnings.filterwarnings("ignore")

In [2]:
df_112 = pd.read_csv(("../Resources/112justice.csv"))
df_112.tail()

,justice,term,direction,precedentAlteration,issueArea,lawType,caseOriginState
937,112,2017,1,0,1,2,55
938,112,2017,1,0,9,9,0
939,112,2017,2,0,2,2,0
940,112,2017,2,0,2,6,0
941,112,2017,1,0,1,6,0


In [3]:
target_112 = df_112["direction"]
target_names = ["conservative", "liberal"]

In [4]:
data_112 = df_112.drop(["direction", "justice"], axis=1)
data_112.head()

,term,precedentAlteration,issueArea,lawType,caseOriginState
0,2005,0,8,6,0
1,2005,0,2,3,0
2,2005,0,9,0,0
3,2005,0,2,3,0
4,2005,0,1,6,0


In [5]:
term_df = pd.get_dummies(data_112["term"], prefix="term")
precAlt_df = pd.get_dummies(data_112["precedentAlteration"], prefix="precedentAlteration")
issueArea_df = pd.get_dummies(data_112["issueArea"], prefix="issueArea")
lawType_df = pd.get_dummies(data_112["lawType"], prefix="lawType")
caseOriginState_df = pd.get_dummies(data_112["caseOriginState"], prefix="caseOriginState")

In [6]:
oneHot_features = pd.concat([term_df, precAlt_df], axis=1)
oneHot_features = pd.concat([oneHot_features, issueArea_df], axis=1)
oneHot_features = pd.concat([oneHot_features, lawType_df], axis=1)
oneHot_features = pd.concat([oneHot_features, caseOriginState_df], axis=1)
oneHot_features.head()

,term_2005,term_2006,term_2007,term_2008,term_2009,term_2010,term_2011,term_2012,term_2013,term_2014,...,caseOriginState_48,caseOriginState_49,caseOriginState_50,caseOriginState_51,caseOriginState_52,caseOriginState_53,caseOriginState_55,caseOriginState_56,caseOriginState_57,caseOriginState_58
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = oneHot_features
feature_names = oneHot_features.columns
y = target_112

### Evaluating different classifiers

In [8]:
clf_lr = LogisticRegression()
clf_rf = RandomForestClassifier()
clf_nb = GaussianNB()
clf_svm = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf_lr, clf_rf, clf_nb, clf_svm], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

5-fold cross validation:

F1: 0.758 [Logistic Regression]
F1: 0.725 [Random Forest]
F1: 0.259 [Naive Bayes]
F1: 0.771 [Linear SVM]


In [9]:
# using all but naive bayes (lowest score) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'hard')

In [10]:
v_clf_soft = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'soft')

In [11]:
new_labels = ["Logistic Regression", "Random Forest", "Linear SVM", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf_lr, clf_rf, clf_svm, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.758 [Logistic Regression]
F1: 0.730 [Random Forest]
F1: 0.771 [Linear SVM]
F1: 0.772 [Voting_Classifier_Hard]
F1: 0.767 [Voting_Classifier_Soft]


### SVM Model Training

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [14]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.691
Test Acc: 0.619


In [15]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.63      0.94      0.75       116
     liberal       0.53      0.11      0.18        73

   micro avg       0.62      0.62      0.62       189
   macro avg       0.58      0.52      0.47       189
weighted avg       0.59      0.62      0.53       189



In [16]:
print(confusion_matrix(predictions, y_test))
predictions[:15], y_test[:15].ravel()

[[109  65]
 [  7   8]]


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1], dtype=int64),
 array([2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2], dtype=int64))

### Save Model and Confirm Load¶

In [17]:
import pickle

filename = '112_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [18]:
#Load Model from Disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6190476190476191
